In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import socket as socket
import os as os
import sys as sys
import h5py
import matplotlib.cm as cm
import itertools as it
import multiprocessing as mp
import allel

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/"  # The Path on Harvard Cluster
    sys.path.append("/n/groups/reich/hringbauer/git/hapBLOCK/python3/")

elif socket_name.startswith("bionc"):
    print("Leipzig Cluster detected!")
    path = "/mnt/archgen/users/hringbauer/"
    sys.path.append("/mnt/archgen/users/hringbauer/git/hapBLOCK/python3/")
    sys.path.insert(0, "/mnt/archgen/users/hringbauer/git/HAPSBURG/package/")
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

from hapsburg.PackagesSupport.h5_python.h5_functions import merge_in_ld_map
from IO.h5_modify import merge_in_af, get_af, get_af1000G, lift_af, save_h5

bionc21
Leipzig Cluster detected!
/mnt/archgen/users/hringbauer
CPU Count: 40


# Functions Harvard

In [15]:
def save_1240kmarkers(snp1240k_path="", marker_path="", ch=0):
    """Save all 1240 Markers of .snp eigenstrat file.
    to marker_path.
    ch: Chromosome. If null filter all of them"""
    df_snp = pd.read_csv(snp1240k_path, header=None, sep=r"\s+", engine="python")
    df_snp.columns = ["SNP", "chr", "map", "pos", "ref", "alt"]
    if ch>0:
        df_snp = df_snp[df_snp["chr"] == ch]
    print(f"Loaded {len(df_snp)} Chr.{ch} SNPs.")

    df_save = df_snp[["chr", "pos"]]
    df_save.to_csv(marker_path, sep="\t", header=None, index=False)
    print(f"Saved {len(df_save)} 1240k Markers on Chr. {ch} to {marker_path}")
    
def save_1240_1000g_kmarkers(ch=3, snp_path="", marker_path=""):
    """Save all 1240 and 1000G Markers of .snp eigenstrat file.
    to marker_path. Loads Ali Path file
    snp_path: Where to find the SNPs plus their types"""
    df1=pd.read_csv(snp_path, sep="\t", header=None)
    df1.columns=["chr", "pos", "pos1", "ref", "alt", "type"]
    print(f"Loaded {len(df1)} SNPs on Chr. {ch}")
    df2 = df1[df1["type"]=="1kg+1240k"]
    print(f"Loaded {len(df2)} Chr.{ch} SNPs in both 1240K and 1000G.")
    df_save = df2[["chr", "pos1"]]
    df_save.to_csv(marker_path, sep="\t", header=None, index=False)
    print(f"Saved {len(df_save)} 1240k+1000G Markers on Chr. {ch} to {marker_path}")
    
def bctools_filter_vcf(in_vcf_path="", out_vcf_path="", marker_path=""):
    """Same as PLINK, but with bcftools and directly via Marker Positions.
    filter_iids: Whether to use the .csv with Indivdiduals to extract"""
    !bcftools view -Oz -o $out_vcf_path -T $marker_path -M2 -v snps $in_vcf_path
    print("Finished BCF tools filtering.")
    
def bctools_filter_vcf_allvariants(in_vcf_path="", out_vcf_path="", marker_path=""):
    """Same as PLINK, but with bcftools and directly via Marker Positions.
    filter_iids: Whether to use the .csv with Indivdiduals to extract"""
    !bcftools view -Oz -o $out_vcf_path -T $marker_path -v snps $in_vcf_path
    print("Finished BCF tools filtering.")
    
def merge_vcfs(in_vcf_paths=[], out_vcf_path=""):
    """Merges Set of VCFs into one VCF. 
    in_vcf_paths: List of VCFs to merge
    out_vcf_path: Output of VCF"""
    paths_merge = " ".join(in_vcf_paths)
    !bcftools concat -n -o $out_vcf_path $paths_merge
    print("Finished BCF tools filtering.")
    
##############################################################
### Function Identical to vcf_to_hdf5.py in cluster/ folder

def vcf_to_1240K_hdf(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf",
                     path_vcf = "./data/vcf/1240k_v43/ch3.vcf.gz",
                     path_h5 = "./data/hdf5/1240k_v43/ch3.h5",
                     marker_path="./data/filters/ho_snps_bcftools_ch3.csv",
                     map_path="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                     ch=3):
    """Convert Ali's vcf to 1240K hdf5. 
    If marker_path empty, no SNP filtering done.
    If map_path empty, no genetic map is merged in.
    """ 
    if len(marker_path)>0:
        bctools_filter_vcf(in_vcf_path = in_vcf_path,
                           out_vcf_path= path_vcf,
                           marker_path = marker_path)
    else: path_vcf = in_vcf_path # Use the unfiltered input in next step
    print("Finished downsampling to 1240K")
    
    allel.vcf_to_hdf5(input=path_vcf, output=path_h5, 
                  fields = ['variants/*', 'calldata/*', "samples"], compression="gzip") # Do the conversion to hdf5. Takes hours
    print("Finished conversion to hdf5!")
    
    if len(map_path)>0:
        merge_in_ld_map(path_h5=path_h5, 
                    path_snp1240k=map_path,
                    chs=[ch])
        
        
########################################################
### New Functions for Leipzig Processing

def get_paths_ch(path_bcfs="", ch=3):
    """Return all unique iid paths of bcfs with chromosome ch."""
    files = os.listdir(path_bcfs)
    iids = [f.split(".")[0] for f in files]
    k = len(set(iids)) 
    print(f"{k} unique iids ")
    
    path_chs = [f for f in files if (f"chr{ch}." in f)]
    k1 = len(path_chs) 
    print(f"{k1} iids for Chr. {ch} found")
    assert(len(path_chs)==k)
    paths_full = [os.path.join(path_bcfs, p) for p in path_chs]
    return paths_full

def merge_grg_bcfs(path_bcfs = "/mnt/archgen/users/childebayeva/GRG/GLIMPSE_MERGED_1240K",
               out_path = "/mnt/archgen/users/hringbauer/data/GRG/bcf_allv1/", ch = 3,
               index = True):
    """Merge imputed bcf files from Gurgy. 
    Output: one combined bcf file in out_path.
    Takes about 5 min for long Chromosome"""
    out_path = f"{out_path}ch{ch}.bcf"
    paths_full = get_paths_ch(path_bcfs, ch=ch)

    vcfs = " ".join(paths_full)  # Get all vcfs in space seperated list

    ### Index Files: Takes about 1min for 100 samples
    if index:
        for p in paths_full[:]:
            !bcftools index $p
    print("Merging bcfs...")
    !bcftools merge -o $out_path $vcfs # Takes ~30 s for long Chromosome
    print(f"Fnished Merging. Output @ {out_path}")
    
def process_h5_ainash(ch = 3, h5_path="", h5_path_save=""):
    """Merges H5 that Ainash created, by bringing together
    GT and GP fields into one framework"""
    with h5py.File(h5_path, "r") as f:  # Load for Sanity Check. See below!
        samples = f["samples"][:]
        samples = [".".join(s.split(".")[:2]) for s in samples[::2]]

        save_h5(gt=f["calldata/GT"][:,1::2,:],
                gp = f["calldata/GP"][:,::2,:],
                ad =  [], ad_group=False,
                ref=f["variants/REF"][:], alt=f["variants/ALT"][:,0],
                pos=f["variants/POS"][:], rec=f["variants/MAP"][:],
                samples=samples, path=h5_path_save,
                compression='gzip', gt_type='int8')

### Transform individual .bcfs to summary HDF5
Takes about 150 Seconds for long chromosome

In [16]:
%%time

for ch in range(1,23):
    ### Merge the Gurgy Data into one bcf
    merge_grg_bcfs(path_bcfs = "/mnt/archgen/users/childebayeva/GRG/GLIMPSE_MERGED_1240K",
               out_path = "/mnt/archgen/users/hringbauer/data/GRG/bcf_allv1/", ch = ch,
               index = True)
    
    in_vcf_path = f"/mnt/archgen/users/hringbauer/data/GRG/bcf_allv1/ch{ch}.bcf"
    path_h5 = f"/mnt/archgen/users/hringbauer/data/GRG/hdf5/raw/ch{ch}.h5"
    path_h5_save = f"/mnt/archgen/users/hringbauer/data/GRG/hdf5/processed/ch{ch}.h5"
    map_path = f"/mnt/archgen/users/hringbauer/data/MinMyc.snp"
    
    vcf_to_1240K_hdf(in_vcf_path = in_vcf_path, path_vcf = "", path_h5=path_h5,
                     marker_path = "", map_path = map_path, ch=ch)
    
    ### Create processed HDF5
    process_h5_ainash(h5_path=path_h5, h5_path_save=path_h5_save)

    print(f"Finished running chromosome {ch}. GZ")

94 unique iids 
94 iids for Chr. 1 found
Merging bcfs...
Fnished Merging. Output @ /mnt/archgen/users/hringbauer/data/GRG/bcf_allv1/ch1.bcf
Finished downsampling to 1240K
Finished conversion to hdf5!
Lifting LD Map from eigenstrat to HDF5...
Loaded 89147 variants.
Loaded 188 individuals.
Loaded 93166 Chr.1 1240K SNPs.
Intersection 89147 out of 89147 HDF5 SNPs
Finished Chromosome 1.
Adding map to HDF5...
We did it. Finished.
Successfully saved 94 individuals to: /mnt/archgen/users/hringbauer/data/GRG/hdf5/processed/ch1.h5
Finished running chromosome 1. GZ
94 unique iids 
94 iids for Chr. 2 found
Merging bcfs...
Fnished Merging. Output @ /mnt/archgen/users/hringbauer/data/GRG/bcf_allv1/ch2.bcf
Finished downsampling to 1240K
Finished conversion to hdf5!
Lifting LD Map from eigenstrat to HDF5...
Loaded 94239 variants.
Loaded 188 individuals.
Loaded 98657 Chr.2 1240K SNPs.
Intersection 94239 out of 94239 HDF5 SNPs
Finished Chromosome 2.
Adding map to HDF5...
We did it. Finished.
Successfull

# Area 51

In [ ]:
### Some random test lines
p1 = os.path.join(path_bcfs, out_path)
!bcftools query -l $p1 | wc -l
##!bcftools query -f '%POS\n' $p1 | wc -l
#!bcftools view $p1

### Test HDF5 that was created above

In [10]:
f = h5py.File("/mnt/archgen/users/hringbauer/data/GRG/hdf5/processed/ch3.h5", "r")  # Load for Sanity Check. See below!

print(list(f)) # Gives highest level folder
print(list(f["calldata"]))
print(list(f["variants"]))
print(np.shape(f["calldata/GT"]))
gt1 = f["calldata/GT"][:10,0,:] # Gets first ten loci for individual 0
#print(np.shape(f["calddata/GT"]))
f.close()

print(gt1)

['calldata', 'samples', 'variants']
['GP', 'GT']
['ALT', 'MAP', 'POS', 'REF']
(77652, 94, 2)
[[0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


(77652, 94, 3)

In [11]:
f.close()

In [7]:
f["samples"][:]

array(['GRG089.A01', 'GRG053.A01', 'GRG088.A01', 'GRG043.A01',
       'GRG090.A01', 'GRG074.A01', 'GRG028.A01', 'GRG097.A01',
       'GRG064.A01', 'GRG108.A01', 'GRG087.A01', 'GRG009.B01',
       'GRG067.A01', 'GRG032.A01', 'GRG034.A01', 'GRG037.A01',
       'GRG019.A01', 'GRG080.A01', 'GRG063.A01', 'GRG048.A01',
       'GRG071.A01', 'GRG109.A01', 'GRG014.A01', 'GRG107.A01',
       'GRG047.A01', 'GRG077.A01', 'GRG073.A01', 'GRG035.A01',
       'GRG095.A01', 'GRG033.A01', 'GRG016.A01', 'GRG054.B01',
       'GRG086.A01', 'GRG011.A01', 'GRG022.A01', 'GRG025.A01',
       'GRG052.A01', 'GRG023.A01', 'GRG091.A01', 'GRG004.A01',
       'GRG059.A01', 'GRG029.A01', 'GRG024.A01', 'GRG076.A01',
       'GRG075.A01', 'GRG017.A01', 'GRG066.A01', 'GRG046.A01',
       'GRG051.B01', 'GRG103.A01', 'GRG006.B01', 'GRG030.A01',
       'GRG018.A01', 'GRG039.A01', 'GRG005.B01', 'GRG056.A01',
       'GRG060.B01', 'GRG045.B01', 'GRG065.A01', 'GRG096.A01',
       'GRG008.A01', 'GRG068.A01', 'GRG010.A01', 'GRG05